In [ ]:
import pandas as pd
import sys
from pathlib import Path
import numpy as np
from itertools import product
from tqdm import tqdm
# sys.path.append('/data/gpfs/projects/punim1395/workspace/DeepCTR-Torch')
# from ..deepctr_torch.layers.utils import slice_arrays
!pwd

In [333]:
meta_path = Path('../small_subsets/Movies_and_TV/\
meta_Movies_and_TV_valid.csv')
meta_Movies_and_TV = pd.read_csv(meta_path).iloc[:, 1:]

/home/liangchenl/.conda/envs/jupyterlab-debugger/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
meta_Movies_and_TV.head()

In [ ]:
rating_path = Path('../small_subsets/Movies_and_TV/\
ratings_Movies_and_TV.csv')
ratings = pd.read_csv(rating_path, header=None, names=['reviewerID', 'asin', 'overall', 'unixReviewTime'])

In [ ]:
ratings.tail()

In [ ]:
ratings_meta_Movies_and_TV = ratings.merge(meta_Movies_and_TV, how='left', on='asin')

In [ ]:
ratings_meta_Movies_and_TV.head()

In [ ]:
ratings_meta_Movies_and_TV_valid= ratings_meta_Movies_and_TV.dropna(subset=['path'])

In [ ]:
ratings_meta_Movies_and_TV_valid.tail()

The output csv for the ratings_meta_Movies_and_TV_valid

In [5]:
outpath = Path('../small_subsets/ratings_meta_Movies_and_TV_valid.csv/ratings_meta_Movies_and_TV_valid.csv')

In [ ]:
ratings_meta_Movies_and_TV_valid.to_csv(outpath)

The output csv for the ratings_meta_Movies_and_TV_valid with valid price

In [9]:
outpath = Path('../small_subsets/Movies_and_TV/ratings_meta_Movies_and_TV_valid_w_price.csv')
outpath_noname = outpath.parent

In [ ]:
ratings_meta_Movies_and_TV_valid_w_price = ratings_meta_Movies_and_TV_valid.dropna(subset=['price'])

In [ ]:
ratings_meta_Movies_and_TV_valid_w_price.head()

In [ ]:
ratings_meta_Movies_and_TV_valid_w_price.to_csv(outpath)

In [ ]:
ratings_meta_Movies_and_TV_valid_random = ratings_meta_Movies_and_TV_valid.sample(frac=1)

In [ ]:
val_ratio = 0.1
test_ratio = 0.1

In [ ]:
table = ratings_meta_Movies_and_TV_valid_random

In [ ]:
len(table)

In [ ]:
if hasattr(table, 'shape'):
    val_split_at = int(table.shape[0] * (1. - val_ratio-test_ratio))
    test_split_at = int(table.shape[0] * (1. - test_ratio))
else:
    val_split_at = int(len(table) * (1. - val_ratio-test_ratio))
    test_split_at = int(len(table) * (1. - test_ratio))

In [ ]:
table_train = table[:val_split_at]
table_val = table[val_split_at:test_split_at]
table_test = table[test_split_at:]

In [ ]:
table_val = table[val_split_at:test_split_at]

In [ ]:
len(table_train)

In [ ]:
len(table_val)

In [ ]:
len(table_test)

In [ ]:
table_train.to_csv(outpath_noname/'ratings_meta_Movies_and_TV_valid_random_08_train.csv') # save the 0.8 of ratings_movies_poster_users_train for training and fair compare

In [ ]:
table_val.to_csv(outpath_noname/'ratings_meta_Movies_and_TV_valid_random_01_val.csv') # save the 0.1 of ratings_movies_poster_users_train for validation and fair compare

In [ ]:
table_test.to_csv(outpath_noname/'ratings_meta_Movies_and_TV_valid_random_01_test.csv') # save the 0.1 of ratings_movies_poster_users_train for validation and fair compare

In [ ]:
outpath = Path('../small_subsets/Movies_and_TV/ratings_meta_Movies_and_TV_valid_w_price.csv')
outpath_noname = outpath.parent

In [ ]:
table_train = pd.read_csv(outpath_noname/'ratings_meta_Movies_and_TV_valid_random_08_train.csv')

In [ ]:
table_val = pd.read_csv(outpath_noname/'ratings_meta_Movies_and_TV_valid_random_01_val.csv') 

In [ ]:
table_test = pd.read_csv(outpath_noname/'ratings_meta_Movies_and_TV_valid_random_01_test.csv')

In [ ]:
table_all = pd.concat([table_train, table_val, table_test])

In [21]:
num_pos = len(table_all[table_all['Binary_rating'] == True]); num_pos

3953154

In [22]:
num_neg = len(table_all[table_all['Binary_rating'] == False]); num_neg

559052

In [23]:
num_diff = num_pos - num_neg

In [24]:
num_diff

3394102

In [26]:
rID_all = np.unique(table_all['reviewerID'])
asin_all = np.unique(table_all['asin'])

In [130]:
num_diff = num_pos - num_neg
num_table_all_balance = len(table_all)+num_diff
ra_all = np.column_stack((np.array(table_all['reviewerID']),
                          np.array(table_all['asin'])))
ra_all = ra_all.astype(str)
ra_all_balance = ra_all
while True:
    rID_extra_index = np.random.choice(len(rID_all), num_diff)
    rID_extra = rID_all[rID_extra_index]
    rID_extra = rID_extra.astype(str)
    asin_extra_index = np.random.choice(len(asin_all), num_diff)
    asin_extra = asin_all[asin_extra_index]
    asin_extra = asin_extra.astype(str)
    ra_extra_index = np.column_stack((rID_extra_index, asin_extra_index))
    ra_extra = np.column_stack((rID_extra, asin_extra))
    ra_all_balance = np.concatenate((ra_all_balance, ra_extra))
    ra_all_balance, ra_index = np.unique(ra_all_balance, axis=0, return_index=True)
    ra_all_balance = ra_all_balance[np.argsort(ra_index)]
    if len(ra_all_balance) < num_table_all_balance:
        num_diff = num_table_all_balance - len(ra_all_balance)
    else:
        break

In [132]:
sample_extra = np.array([0.]*(num_pos - num_neg))
overall_col = np.array(table_all['overall'])
overall_col = np.concatenate([overall_col, sample_extra])
overall_col = overall_col.astype('int')

In [158]:
rao_all_balance = ra_all_balance

In [160]:
rao_all_balance = np.column_stack((rao_all_balance, overall_col.astype(np.int8)))

In [171]:
df = pd.DataFrame(rao_all_balance, columns = ['reviewerID', 'asin', 'overall'])

In [182]:
df['Binary_rating'] = df['overall']c.astype(int)>2

In [192]:
df.to_csv(rao_all_balance_path)

In [182]:
df['Binary_rating'] = df['overall'].astype(int)>2

In [ ]:
df_random_raob = df.sample(frac=1) 
# raob means rID asin overall Binary_rating

In [ ]:
meta_path = Path('../small_subsets/Movies_and_TV/\
meta_Movies_and_TV_valid.csv')
meta_Movies_and_TV = pd.read_csv(meta_path).iloc[:, 1:]
ap_Movies_and_TV = meta_Movies_and_TV[['asin', 'path']]

In [ ]:
df_random = pd.merge(df_random_raob, ap_Movies_and_TV, how='left', on='asin')

In [ ]:
val_ratio = 0.1
test_ratio = 0.1

In [ ]:
if hasattr(df_random, 'shape'):
    val_split_at = int(df_random.shape[0] * (1. - val_ratio-test_ratio))
    test_split_at = int(df_random.shape[0] * (1. - test_ratio))
else:
    val_split_at = int(len(df_random) * (1. - val_ratio-test_ratio))
    test_split_at = int(len(df_random) * (1. - test_ratio))

In [ ]:
df_random_train = df_random[:val_split_at]
df_random_val = df_random[val_split_at:test_split_at]
df_random_test = df_random[test_split_at:]

In [ ]:
df_random_train.to_csv(outpath_noname/'rao_Movies_and_TV_random_08_train_binary_balance.csv')
# save the 0.8 of rID asin and overall train for training and fair compare

In [ ]:
df_random_val.to_csv(outpath_noname/'rao_Movies_and_TV_random_01_val_binary_balance.csv') 
# save the 0.1 of rID asin and overall val for validation and fair compare

In [ ]:
df_random_test.to_csv(outpath_noname/'rao_Movies_and_TV_random_01_test_binary_balance.csv') 
# save the 0.1 of rID asin and overall test for test and fair compare